# Recomendation Systems #

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [26]:
spark = SparkSession.builder.appName("RecommendationSystemData").getOrCreate()
data = [
    (1, 101, 4.5, "2023-01-01 10:00:00"),
    (2, 105, 3.0, "2023-01-01 11:15:00"),
    (1, 106, 5.0, "2023-01-02 09:45:00"),
    (3, 103, 2.0, "2023-01-02 15:20:00"),
    (2, 102, 4.0, "2023-01-03 13:35:00"),
    (3, 104, 4.0, "2023-01-03 17:00:00"),
    (1, 105, 3.5, "2023-01-04 08:00:00"),
    (4, 101, 2.0, "2023-01-04 10:00:00"),
    (4, 102, 3.5, "2023-01-05 12:15:00"),
    (5, 103, 5.0, "2023-01-05 14:30:00"),
    (6, 104, 4.5, "2023-01-06 16:00:00"),
    (5, 106, 2.0, "2023-01-07 09:15:00"),
    (7, 107, 4.0, "2023-01-07 11:45:00"),
    (8, 108, 3.5, "2023-01-08 13:25:00"),
    (9, 109, 4.0, "2023-01-08 15:40:00"),
    (10, 110, 5.0, "2023-01-09 08:50:00"),
    (1, 111, 3.0, "2023-01-10 10:20:00"),
    (2, 112, 4.0, "2023-01-10 12:30:00"),
    (3, 113, 2.5, "2023-01-11 09:00:00"),
    (4, 114, 3.5, "2023-01-11 14:45:00"),
    (5, 115, 4.5, "2023-01-12 15:15:00"),
    (6, 116, 2.0, "2023-01-12 17:30:00"),
    (7, 117, 5.0, "2023-01-13 18:00:00"),
    (8, 118, 4.0, "2023-01-14 10:30:00"),
    (9, 119, 3.0, "2023-01-15 11:00:00"),
    (10, 120, 4.0, "2023-01-15 12:15:00"),
    (1, 121, 3.5, "2023-01-16 08:20:00"),
    (2, 122, 2.5, "2023-01-16 14:30:00"),
    (3, 123, 5.0, "2023-01-17 09:50:00"),
    (4, 124, 4.5, "2023-01-18 10:10:00"),
    (5, 125, 3.0, "2023-01-19 12:20:00"),
    (6, 126, 2.5, "2023-01-20 15:35:00"),
    (7, 127, 4.0, "2023-01-21 16:00:00"),
    (8, 128, 5.0, "2023-01-22 10:45:00"),
    (9, 129, 3.5, "2023-01-23 11:10:00"),
    (10, 130, 4.0, "2023-01-24 12:30:00"),
    (1, 131, 2.0, "2023-01-25 14:50:00"),
    (2, 132, 3.5, "2023-01-26 09:15:00"),
    (3, 133, 4.5, "2023-01-27 10:00:00"),
    (4, 134, 3.0, "2023-01-28 13:30:00"),
    (5, 135, 4.0, "2023-01-29 15:00:00"),
    (6, 136, 2.5, "2023-01-30 17:20:00"),
    (7, 137, 5.0, "2023-01-31 08:00:00"),
    (8, 138, 3.5, "2023-02-01 09:45:00"),
    (9, 139, 4.0, "2023-02-02 11:00:00"),
    (10, 140, 3.0, "2023-02-03 14:00:00"),
]

# The DataFrame creation code would remain the same

columns = ["user_id", "item_id", "rating", "timestamp"]
df = spark.createDataFrame(data, schema=columns)
df.printSchema()
df.show()


root
 |-- user_id: long (nullable = true)
 |-- item_id: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: string (nullable = true)

+-------+-------+------+-------------------+
|user_id|item_id|rating|          timestamp|
+-------+-------+------+-------------------+
|      1|    101|   4.5|2023-01-01 10:00:00|
|      2|    105|   3.0|2023-01-01 11:15:00|
|      1|    106|   5.0|2023-01-02 09:45:00|
|      3|    103|   2.0|2023-01-02 15:20:00|
|      2|    102|   4.0|2023-01-03 13:35:00|
|      3|    104|   4.0|2023-01-03 17:00:00|
|      1|    105|   3.5|2023-01-04 08:00:00|
|      4|    101|   2.0|2023-01-04 10:00:00|
|      4|    102|   3.5|2023-01-05 12:15:00|
|      5|    103|   5.0|2023-01-05 14:30:00|
|      6|    104|   4.5|2023-01-06 16:00:00|
|      5|    106|   2.0|2023-01-07 09:15:00|
|      7|    107|   4.0|2023-01-07 11:45:00|
|      8|    108|   3.5|2023-01-08 13:25:00|
|      9|    109|   4.0|2023-01-08 15:40:00|
|     10|    110|   5.0|2023-01

In [27]:
df = df.drop("timestamp")
(train_data, test_data) = df.randomSplit([0.8, 0.2], seed=42)
als = ALS(userCol="user_id",
          itemCol="item_id",
          ratingCol="rating",
          rank=10,maxIter=5,
          regParam=0.01,
          coldStartStrategy="drop")
model = als.fit(train_data)
predictions = model.transform(test_data)
predictions = predictions.dropna(subset=["prediction"])
if predictions.count() == 0:
    print("No predictions available for evaluation.")
else:
    # Evaluate the model using RMSE
    evaluator_rmse = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
    rmse = evaluator_rmse.evaluate(predictions)
    print(f"Root-mean-square error (RMSE) = {rmse}")

    # Evaluate the model using MAE
    evaluator_mae = RegressionEvaluator(metricName="mae", labelCol="rating", predictionCol="prediction")
    mae = evaluator_mae.evaluate(predictions)
    print(f"Mean Absolute Error (MAE) = {mae}")

    # Evaluate the model using R-squared
    evaluator_r2 = RegressionEvaluator(metricName="r2", labelCol="rating", predictionCol="prediction")
    r2 = evaluator_r2.evaluate(predictions)
    print(f"R-squared = {r2}")

    # Show the predictions
    predictions.show()


24/11/05 13:07:33 ERROR Instrumentation: java.lang.IllegalArgumentException: Userid does not exist. Available: user_id, item_id, rating
	at org.apache.spark.sql.types.StructType.$anonfun$apply$1(StructType.scala:282)
	at scala.collection.immutable.Map$Map3.getOrElse(Map.scala:336)
	at org.apache.spark.sql.types.StructType.apply(StructType.scala:281)
	at org.apache.spark.ml.util.SchemaUtils$.checkNumericType(SchemaUtils.scala:75)
	at org.apache.spark.ml.recommendation.ALSParams.validateAndTransformSchema(ALS.scala:259)
	at org.apache.spark.ml.recommendation.ALSParams.validateAndTransformSchema$(ALS.scala:257)
	at org.apache.spark.ml.recommendation.ALS.validateAndTransformSchema(ALS.scala:616)
	at org.apache.spark.ml.recommendation.ALS.transformSchema(ALS.scala:753)
	at org.apache.spark.ml.recommendation.ALS.$anonfun$fit$1(ALS.scala:715)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:191)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache

IllegalArgumentException: Userid does not exist. Available: user_id, item_id, rating